In [1]:
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import gc

In [3]:
df = pd.read_csv('E:/Datasets/validation-annotations-human-imagelabels.csv', usecols=[0,2,3])

In [4]:
df = df[df.Confidence == 1]

In [5]:
import numpy  as np

In [15]:
classes = np.array(['/m/01g317', '/m/09j2d', '/m/04yx4', '/m/0dzct', '/m/07j7r', '/m/05s2s', '/m/03bt1vf', '/m/07yv9', '/m/0cgh4', '/m/01prls', '/m/09j5n', '/m/0jbk', '/m/0k4j', '/m/05r655', '/m/02wbm', '/m/0c9ph5', '/m/083wq', '/m/0c_jw', '/m/03jm5', '/m/0d4v4'])

In [7]:
li = []
for i in classes:
    li.append(df[df.LabelName == i])

In [8]:
df = pd.concat(li).sample(frac=1).reset_index(drop=True)

In [9]:
del li
gc.collect()
df.head()

,ImageID,LabelName,Confidence
0,a9371dcd7d3087d9,/m/083wq,1
1,cd5019c2a2ad3b18,/m/0jbk,1
2,a29e776816254a56,/m/05s2s,1
3,bda98befc7e77a86,/m/07yv9,1
4,35b0cb91a7db8a79,/m/01g317,1


In [10]:
labels = df.LabelName.tolist()
Imageid = df.ImageID.values
print(len(df))

68867


In [11]:
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model, Sequential
from tqdm import tqdm_notebook
from tqdm import tqdm
from keras.optimizers import Adam

In [12]:
import sys 
from PIL import Image 

In [13]:
X_train = [np.array(load_img(r'E:\Datasets\validation\{}.jpg'.format(i),target_size=(139,139), grayscale=False))/255 for i in tqdm(Imageid[10000:20000])]

100%|██████████| 10000/10000 [16:45<00:00,  2.18it/s]


In [17]:
classes = classes.tolist()
Y_train, Y_Val = [], []
for i in tqdm(labels[10000:20000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_train.append(temp)
    del temp
for i in tqdm(labels[:2000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_Val.append(temp)
    del temp
Y_train = np.array(Y_train)
Y_Val = np.array(Y_Val)
gc.collect(), Y_train[0]

100%|██████████| 2000/2000 [00:00<00:00, 117718.33it/s]


(280,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0.]))

In [20]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

#x = myModel.output
#x = GlobalAveragePooling2D()(x)
#x = Dense(100, activation='relu')(x)
#predictions = Dense(20, activation='softmax')(x)

#model = Model(inputs=myModel.input, outputs=predictions)


#print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
X_train = np.array(X_train).reshape((10000,139,139,3))
X_Val = np.array(X_Val).reshape((2000,139,139,3))

In [ ]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

In [ ]:
plotinfo=model.fit(X_train, Y_train, validation_data=(X_Val,Y_Val), batch_size=50, epochs=5, verbose=1)